# Initialize

In [ ]:
import os
from time import *

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys 
sys.path.append('..\\')
from scresonators.fit_resonator.ana_resonator import ResonatorFitter
from scresonators.fit_resonator.ana_resonator import ResonatorData
import csv
from datetime import datetime
from scresonators.measurement import vna_measurement
from scresonators.measurement import helpers
from scresonators.measurement import vna_scan
%load_ext autoreload
%autoreload 2

helpers.config_figs()

import scresonators.measurement.vna_measurement
from scresonators.measurement.ZNB import ZNB20

VNA = ZNB20(address='10.108.30.65') # Stanford
#VNA = ZNB20(address='192.168.137.84') # SLAC

## Daily measurement settings
warm_att = 0 #dB
cold_att = 50

spar = 's21'
base_path = r'G:/My Drive/Projects/Materials/Data/250518-ResSilTa1_cooldown2p/'

# spar = 's43'
# base_path = r'G:/My Drive/Projects/Materials/Data/250518-ResSilTa3_cooldown2'

if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
VNA.reset()

# Single power scans

## Single scan

In [ ]:
expt_path = base_path
scan_def = {'freq_center':6.859e9, 
            'span':5e6,  
            'bandwidth':10000, 
            'power':-10, 
            'npoints':10000, 
            'averages':4}
#file_name = 'res_' + str(freq_center)[:7] + '_' + str(power)[1:] + 'dbm'
file_name = 'search'
data=vna_scan.do_vna_scan(VNA, file_name, expt_path, scan_def, spar)

# 5.986213520, 6.264641450, 6.418856885, 6.570072000, 6.8067, 7.085925, 7.1553305, 7.3837534, 7.589974, 7.754795

## Check resonances at one power

In [ ]:
%matplotlib inline
expt_path = base_path
npoints = 150
span=0.15e6
power = -10

freqs = np.array([6.0175e+09, 6.3510e+09, 6.7120e+09, 7.0296e+09, 7.1744e+09, 7.5467e+09])
freqs =np.array([6.7278e+09, 6.9235e+09, 7.0342e+09])
bandwidth=50
avgs=1
for freq_center in freqs:     
    scan_def = {'freq_center':freq_center, 'span':span,  'bandwidth':bandwidth, 'power':power, 'npoints':npoints, 'averages':avgs}
    file_name = 'res_' + str(freq_center)[:7] + '_' + str(power)[1:] + 'dbm'
    vna_scan.do_vna_scan(VNA, file_name, expt_path, scan_def, spar)

In [ ]:
from scipy.stats import linregress

plt.figure()
plt.plot(data['freqs'], np.unwrap(data['phases']))

# Fit the data to a line
slope, intercept, r_value, p_value, std_err = linregress(data['freqs'], np.unwrap(data['phases']))

# Plot the fitted line
plt.plot(data['freqs'], slope * data['freqs'] + intercept, label='Fitted Line', color='red')
plt.legend()
plt.show()
#config.update_config(cfg_path, 'readout', qi, 'phase_const', slope)

In [ ]:
slope

# Run all these cells to do power sweep

In [ ]:
expt_path = base_path + '/Res1_650mK'
os.makedirs(expt_path, exist_ok=True)

## Broad frequency scan looking for resonances 

In [ ]:
#expt_path = base_path
span=50e6
power = 0
avgs = 3
d=[]
bandwidth=5000
freq_min=6.0e9
freq_max = 8.1e9 
nscans = int(np.ceil((freq_max - freq_min)/span))
for i in range(nscans):
    freq_center = freq_min + i*span
    scan_def = {'freq_center':freq_center, 
                'span':span,  
                'bandwidth':bandwidth, 
                'power':power, 
                'npoints':10000, 
                'averages':avgs}
    file_name = 'res_' + str(freq_center)[:7] + '_' + str(power)[1:] + 'dbm'
    d.append(vna_scan.do_vna_scan(VNA, file_name, expt_path, scan_def, spar, plot=False))

In [ ]:
dd = helpers.concat_scans(d)
dd['amps'] = helpers.make_lin(dd['amps'])
plt.figure()
plt.plot(dd['freqs'],dd['amps'])

In [ ]:
dd_db = helpers.concat_scans(d)
plt.figure()
plt.plot(dd_db['freqs'],dd_db['amps'])
plt.xlabel('Frequency (GHz)')
plt.ylabel('Amplitude (dB)')
plt.savefig('amp_vs_freq_3.png')

In [ ]:
data = helpers.find_peaks(dd, 0.003)
print(len(data['coarse_peaks']))

In [ ]:
data['coarse_props']

In [ ]:
freqs = data['coarse_peaks']

In [ ]:
freqs

## Medium width scan checking each resonance 
Saves the minimum value of transmission to res_freqs for narrower scan next. 

In [ ]:
#freqs = np.array([6.0175e+09, 6.3510e+09, 6.7120e+09, 6.8767e+09,7.0296e+09, 7.1744e+09, 7.5467e+09,8.0459e+09])
#freqs =np.array([6.7278e+09, 6.9235e+09, 7.0342e+09,8.0683e+09])
#freqs = np.array([6.7278e+09, 6.9234e+09, 7.0342e+09, 7.3375e+09,
#freqs = data['coarse_peaks']              

# 8.0683e+09]) # res 2
#freqs = np.array([6.0155e+09, 6.3552e+09, 6.7128e+09,6.8589e9, 7.0288e+09, 7.1631e+09, 7.5432e+09,
# 8.0420e+09])
expt_path = base_path
span=2e6
res_freqs =[]
power=-10
temp=0.03

freqs=np.array([6.0155e+09, 6.3554e+09, 6.7129e+09, 7.0289e+09, 7.1632e+09,
       7.5434e+09, 8.0421e+09, 6.859e9])

 #np.array([6.0155e+09, 6.3552e+09, 6.7128e+09, 6.8589e+09, 7.0288e+09, 7.1631e+09, 7.5432e+09, 8.0420e+09])

for freq_center in freqs:
    scan_def = {'freq_center':freq_center, 'span':span,  'bandwidth':1000, 'power':-23, 'npoints':1000, 'averages':2}
    file_name = 'res_' + str(freq_center)[:7] + '_' + str(power)[1:] + 'dbm_' + str(temp) + 'K'
    data=vna_scan.do_vna_scan(VNA, file_name, expt_path, scan_def, spar)
    res_freqs.append(data['freqs'][np.argmin(data['amps'])])

In [ ]:
expt_path = base_path + '/Res1_200mK_2'
os.makedirs(expt_path, exist_ok=True)

In [ ]:
scan_def

## Narrower scan before power sweep

In [ ]:
freq_list =[]
import copy 
scan_def['span']=1e5
scan_def['power']=-10
for res in res_freqs:
    scan_def['freq_center'] = res
    file_name = 'res_' + str(scan_def['freq_center'])[:7] + '_' + str(power)[1:] + 'dbm'
    data=vna_scan.do_vna_scan(VNA, file_name, expt_path, scan_def, spar)
    freq_list.append(data['freqs'][np.argmin(data['amps'])])
res_freqs=copy.deepcopy(freq_list)

In [ ]:
freq_list

# Power sweep

In [ ]:
# # 1
# freq_list = [6017455952.976488,
#  6350950600.30015,
#  6711990820.410205,
#  6876720085.042521,
#  7029601875.937968,
#  7174401175.587793,
#  7546665057.528765,
#  8045895572.786393]
freq_list = [6017575675.136297,
 6351091177.546969,
 6712102126.580842,
 6876862804.329457,
 7029727928.775868,
 7174591126.581392,
 7546820346.301764,
 8046069120.718976]

# freq_list = [6876862804.329457,
#  7029727928.775868,
#  7546820346.301764,
#  8046069120.718976]


freq_list = [6712102126.580842,
 7029727928.775868,]
 # 1
#freq_list = [
# # 6876720085.042521]
slope = -4.1534052061318027e-07 # 1 
#slope=0
#freq_list = [8045895572.786393]
# freq_list =[8068465902.491149]
# 2
#freq_list = [6727757253.626813, 6923469659.829915, 7034221985.992996, 8068307578.789394] # 2 
#freq_list = [8068307578.789394]
# slope = -4.6912118164934637e-07
#freq_list= [7034221985.992996]

# 3
# freq_list = [6015457003.501751,
#  6355251900.950475,
#  6712772761.38069,
#  #6859872161.080541,
#  7028836743.371686,
#  7163106428.2141075,
#  7543248049.024512,
#  8041999124.562282]
 # single, linear 
 # 001

# freq_list=[6015547597.597597,
#  6355356506.506506,
#  6712867517.517517,
#  7028931581.581581,
#  7163240390.39039,
#  7543367617.617618,
#  8042131781.781781,
#  6859011361.3613615]
slope = 0 


for i in range(1):
    custom_config = {"freqs":np.array(freq_list), 
                 "att":65,
                 'type':'single',#'single , lin',
                 'base_path':base_path, 
                 'bandwidth':100,
                 "db_slope":5,
                 'slope':slope,
                 "span_inc":8,
                 'nvals':35,
                 "pow_start":11,
                 "avg_corr":0.04,
                 'spar':spar,
                 'npoints':22, 
                 'pow_inc':-4, 
                 'comment':'res 3, going for it'}
#custom_config = {"freqs":np.array(freq_list), "att":50,'type':'lin','base_path':base_path, 'nvals':21,"pow_start":0}
    # Update default config with custom values
    config = vna_measurement.get_default_power_sweep_config(custom_config)
    results = vna_measurement.power_sweep_v2(config, VNA)


## Fast version

In [ ]:
# 1
freq_list = [6017455952.976488,
 6350950600.30015,
 6711990820.410205,
 6876720085.042521,
 7029601875.937968,
 7174401175.587793,
 7546665057.528765,
 8045895572.786393]
freq_list = [6017575675.136297,
 6351091177.546969,
 6712102126.580842,
 6876862804.329457,
 7029727928.775868,
 7174591126.581392,
 7546820346.301764,
 8046069120.718976]

 # 1
#freq_list = [
# 6876720085.042521]
slope = -4.1534052061318027e-07 # 1 
#slope=0
#freq_list = [8045895572.786393]
# 2
#freq_list = [6727757253.626813, 6923469659.829915, 7034221985.992996, 8068307578.789394] # 2 
#freq_list = [8068307578.789394]
#slope = -4.6912118164934637e-07
#freq_list= [7034221985.992996]

# 3
# freq_list = [6015457003.501751,
#  6355251900.950475,
#  6712772761.38069,
#  #6859872161.080541,
#  7028836743.371686,
#  7163106428.2141075,
#  7543248049.024512,
#  8041999124.562282]
 # single, linear 
 # 001

for i in range(500):
    custom_config = {"freqs":np.array(freq_list), 
                 "att":65,
                 'type':'lin',#'single , lin',
                 'base_path':base_path, 
                 'bandwidth':100,
                 "db_slope":5,
                 'slope':slope,
                 "span_inc":8,
                 'nvals':6,
                 "pow_start":5,
                 "avg_corr":0.1,
                 'spar':spar,
                 'npoints':75, 
                 'pow_inc':-4, 
                 'comment':'cooling down, starting at 900 mK'}
#custom_config = {"freqs":np.array(freq_list), "att":50,'type':'lin','base_path':base_path, 'nvals':21,"pow_start":0}
    # Update default config with custom values
    config = vna_measurement.get_default_power_sweep_config(custom_config)
    results = vna_measurement.power_sweep_v2(config, VNA)
    write_hp_csv(results)


In [ ]:
new_hp_csv(config['base_path'])

In [ ]:


qhp_list = []
for i in range(len(results.measurements)):
    q_int = [results.measurements[i][j].q_internal for j in range(len(results.measurements[i]))]
    qhp = np.max(q_int)
    qhp_list.append(qhp)

plt.figure(figsize=(8, 7))
plt.semilogy(qhp_list, 'o')
qhp_list
# Create an empty CSV file with headers if it doesn't exist
fname = os.path.join(config['base_path'], 'qhp.csv')
 # Adjust headers as needed
with open(fname, mode='a', newline='') as file:
    writer = csv.writer(file)
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    writer.writerow([now] + qhp_list)


## Power sweep just 1

In [ ]:
# 1
freq_list = [7029727828.725843,]

 # 1
#freq_list = [
# 6876720085.042521]
slope = -4.1534052061318027e-07 # 1 
#slope=0
#freq_list = [8045895572.786393]
# 2
#freq_list = [6727757253.626813, 6923469659.829915, 7034221985.992996, 8068307578.789394] # 2 
#freq_list = [8068307578.789394]
#slope = -4.6912118164934637e-07
#freq_list= [7034221985.992996]

# 3
# freq_list = [6015457003.501751,
#  6355251900.950475,
#  6712772761.38069,
#  #6859872161.080541,
#  7028836743.371686,
#  7163106428.2141075,
#  7543248049.024512,
#  8041999124.562282]
 # single, linear 
 # 001
custom_config = {"freqs":np.array(freq_list), 
                 "att":65,
                 'type':'lin',#'single , lin',
                 'base_path':base_path, 
                 'bandwidth':100,
                 "db_slope":7,
                 'slope':slope,
                 "span_inc":8,
                 'nvals':10,
                 "pow_start":10,
                 "avg_corr":0.1,
                 'spar':spar,
                 'npoints':100, 
                 'pow_inc':-4, 
                 'comment':'365 mK, 10 data points, warming'}
#custom_config = {"freqs":np.array(freq_list), "att":50,'type':'lin','base_path':base_path, 'nvals':21,"pow_start":0}
    # Update default config with custom values
config = vna_measurement.get_default_power_sweep_config(custom_config)
print(config)
results = vna_measurement.power_sweep_v2(config, VNA)